# Create memory-mapped HTSeq genomic Array

Get chromosome sizes to use memory mapped mode of HTSeq so it's faster

In [2]:
cd /home/obotvinnik/projects/singlecell_pnms/analysis/htseq_memmap

/oasis/tscc/scratch/obotvinnik/projects/singlecell_pnms/analysis/htseq_memmap


In [4]:
ls -lha

total 12G
drwxr-xr-x  2 obotvinnik yeo-group 33K Jan 28 21:14 ./
drwxr-xr-x 16 obotvinnik yeo-group 33K Jan 27 19:41 ../
-rw-r--r--  1 obotvinnik yeo-group 79G Sep 26  2015 hg19_phastcons_placental_mammal_htseq.pickle


In [6]:
! touch *

In [7]:
ls -lha

total 12G
drwxr-xr-x  2 obotvinnik yeo-group 33K Jan 28 21:14 ./
drwxr-xr-x 16 obotvinnik yeo-group 33K Jan 27 19:41 ../
-rw-r--r--  1 obotvinnik yeo-group 79G Jun 13 06:18 hg19_phastcons_placental_mammal_htseq.pickle


In [9]:
!head hg19_phastcons_placental_mammal_htseq.pickle

cHTSeq._HTSeq
_GenomicArray_unpickle
p1
(I00
S'd'
(dp2
S'chrY'
p3
(dp4
S'.'


In [11]:
import cPickle as pickle
memmap_dir = '/home/obotvinnik/projects/singlecell_pnms/analysis/htseq_memmap'

filename = '{}/hg19_phastcons_placental_mammal_htseq.pickle'.format(memmap_dir)
with open(filename) as f:
    conservation = pickle.load(f)

In [ ]:
interval = 'chr17:41,198,123-41,198,198'

In [13]:
import HTSeq

In [15]:
chromvector = conservation[HTSeq.GenomicInterval('chr17', 41198123, 41198198)]

import numpy as np

np.array([x for x in chromvector.values()])

In [ ]:
interval= 'chr10:79,797,655-79,797,710'

In [23]:
chromvector = conservation[HTSeq.GenomicInterval('chr10', 79797655, 79797710)]

np.array([x for x in chromvector.values()])

array([ 0.991,  0.997,  0.997,  0.996,  0.983,  0.939,  0.939,  0.94 ,
        0.94 ,  0.936,  0.925,  0.921,  0.921,  0.922,  0.914,  0.91 ,
        0.906,  0.874,  0.857,  0.836,  0.844,  0.844,  0.928,  0.959,
        0.991,  0.996,  1.   ,  0.999,  0.994,  0.994,  0.988,  0.947,
        0.941,  0.948,  0.935,  0.924,  0.925,  0.922,  0.941,  0.945,
        0.936,  0.73 ,  0.679,  0.654,  0.493,  0.458,  0.489,  0.971,
        0.992,  1.   ,  1.   ,  1.   ,  1.   ,  1.   ,  1.   ])

In [1]:
import pandas as pd
chromsizes = pd.read_table('/projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes', header=None, index_col=0, squeeze=True)

# Remove all haplotype chromosomes
chromsizes = chromsizes[chromsizes.index.map(lambda x: '_' not in x)]
chromsizes = chromsizes.to_dict()
chromsizes

{'chr1': 249250621,
 'chr10': 135534747,
 'chr11': 135006516,
 'chr12': 133851895,
 'chr13': 115169878,
 'chr14': 107349540,
 'chr15': 102531392,
 'chr16': 90354753,
 'chr17': 81195210,
 'chr18': 78077248,
 'chr19': 59128983,
 'chr2': 243199373,
 'chr20': 63025520,
 'chr21': 48129895,
 'chr22': 51304566,
 'chr3': 198022430,
 'chr4': 191154276,
 'chr5': 180915260,
 'chr6': 171115067,
 'chr7': 159138663,
 'chr8': 146364022,
 'chr9': 141213431,
 'chrM': 16571,
 'chrX': 155270560,
 'chrY': 59373566}

Populating `HTSeq.GenomicArray` took overnight so plan accordingly

In [ ]:
%%time
import HTSeq

wiggle_filename = '/projects/ps-yeolab/genomes/hg19/hg19_phastcons_placental_mammal_space_separated.wig'
wig = HTSeq.WiggleReader(wiggle_filename)

conservation = HTSeq.GenomicArray(chromsizes, stranded=False, typecode='d', storage='memmap', 
                                  memmap_dir='/home/obotvinnik/projects/singlecell_pnms/analysis/htseq_memmap')
for location, score in wig:
    conservation[location] += score

In [ ]:
%%time

import cPickle as pickle
memmap_dir = '/home/obotvinnik/projects/singlecell_pnms/analysis/htseq_memmap'
with open('{}/hg19_phastcons_placental_mammal_htseq.pickle'.format(memmap_dir), 'wb') as f:
    pickle.dump(conservation, f)